In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def sig(x):
  return 1/(1+np.exp(-x))
def stand(X2):
  X1 = []
  for x in X2.T[:]:
    x = (x - x.mean())/(np.std(x)+1)
    X1.append(x)
  X1 = np.array(X1)
  X1 = X1.T
  return X1
def probability(X,w,b):
  prob = sig(np.dot(X,w)+b)
  return prob
def x_transform(x,degrees):
  t = x.copy()
  for i in degrees:
    x = np.append(x,t**i,axis=1)
  return x
def Poly_log(X,y,lr,no_iter,degrees):  
  X = x_transform(X,degrees)
  X = stand(X)
  m,n = X.shape
  w = np.zeros((n,1))
  b = 0
  costs = []
  lsts = [i for i in range(no_iter)]
  e = 1e-5
  for i in range(no_iter):
    h_x = sig(np.dot(X,w) + b)

    cost = -(1/m)*np.sum(y*np.log(h_x+e) + (1-y)*np.log(1-h_x+e))

      
    dw = (1/m)*(np.dot(X.T,(h_x-y)))
      
    db = (1/m)*np.sum((h_x-y),axis=0)

    w -= lr*dw
    b -= lr*db
    #h_x = sig(np.dot(X,w)+b)
    costs.append(cost)
    if i%(no_iter/10) == 0:
      print('cost after',i,'iteration is= ',cost)
  plt.show()
  return w,b
def testing(w,b,Xtest):
  A = np.dot(Xtest,w)+b
  Z = sig(A)
  #Z = Z>0.5
  #Z = np.array(Z,dtype = 'int64')
  return Z
  


In [3]:
df = pd.read_csv('emnist-letters-train.csv')
df3 = pd.read_csv('emnist-letters-train.csv',header = 0)
df3.drop(columns = ['23'],inplace = True)
df1 = pd.read_csv('emnist-letters-train.csv',header = 0)
df1.drop(columns = ['23'],inplace = True)
for i in range(1,27):
  a = list(df['23'] == i)
  for j in range(len(a)):
    a[j] = int(a[j])
  df1[str(i)] = a

In [4]:
X =[]
for i in range(88799):
  X.append(df3.loc[i])
X = np.array(X)
X.shape

(88799, 784)

In [5]:
W =[]
B=[]
for i in range(1,27):
  y = []
  for j in range(X.shape[0]):
    y.append(df1.loc[j,[str(i)]])
  y = np.array(y)
  w,b = Poly_log(X,y,0.7,500,[2])
  W.append(w)
  B.append(b)

cost after 0 iteration is=  0.6931271807599426
cost after 50 iteration is=  0.10271836792355833
cost after 100 iteration is=  0.09759211199036388
cost after 150 iteration is=  0.09506838769504716
cost after 200 iteration is=  0.09313136646311176
cost after 250 iteration is=  0.091546262316994
cost after 300 iteration is=  0.0902222111669597
cost after 350 iteration is=  0.08910419618605474
cost after 400 iteration is=  0.08815382221477677
cost after 450 iteration is=  0.08734212560630686
cost after 0 iteration is=  0.6931271807599426
cost after 50 iteration is=  0.08226119163405375
cost after 100 iteration is=  0.07652852466209611
cost after 150 iteration is=  0.07390674720301676
cost after 200 iteration is=  0.07196604211329953
cost after 250 iteration is=  0.07039512832372088
cost after 300 iteration is=  0.06907906710026732
cost after 350 iteration is=  0.0679569038418257
cost after 400 iteration is=  0.06698908412814092
cost after 450 iteration is=  0.06614689574811342
cost after 0

In [6]:
dftest1 = pd.read_csv('emnist-letters-test.csv')
dftest = pd.read_csv('emnist-letters-test.csv')
dftest.drop(columns = ['1'],inplace  = True)

In [9]:
x =[]
for i in range(14799):
  x.append(dftest.loc[i])
x = np.array(x)
x = x_transform(x,[2])
x = stand(x)
x.shape
ytest = np.array(dftest1['1'])

In [10]:
ypreds = []
W = np.array(W)
B = np.array(B)
for i in range(26):
  ypred = testing(W[i],B[i],x)
  ypreds.append(ypred)
ypreds = np.array(ypreds)
ypreds = ypreds.reshape(26,14799)
outlst =(ypreds).argmax(axis = 0)+1

outlst = np.array(outlst)
outs = pd.DataFrame(outlst)
outs

,0
0,24
1,1
2,17
3,1
4,1
...,...
14794,19
14795,19
14796,19
14797,10


In [11]:
correct = 0
for i in range(x.shape[0]):
  if outlst[i] == ytest[i]:
    correct += 1

In [12]:
(correct/14799)*100

68.49111426447733